In [1]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [77]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [78]:
lines = pd.read_table(r'C:\Users\Hp\Downloads\korean.txt', names=['english', 'korean'])
lines = lines[:8000]
lines.sample(5)

,english,korean
3633,The fact that Tom didn't do that can't be denied.,톰이 그걸 안 했다는 사실은 부인할 수 없는 사실이야.
2492,All men are equal under the law.,모든 사람은 법 아래에 동등해.
3783,You should try to see things from the practica...,실용적 관점으로 보려고 노력해야 할 것 같아.
3671,"Well, you obviously have romantic feelings for...","흠, 너는 확실히 톰에게 연애감정을 가지고 있군."
2839,How long will you be staying there?,거기서 얼마나 기다리고 있을 거야?


In [79]:
lines.shape

(3833, 2)

In [80]:
lines.english = lines.english.apply(lambda x: x.lower())


In [81]:
import re
lines.english = lines.english.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", 'COMMA', x))
lines.korean = lines.korean.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", 'COMMA', x))

In [82]:
import string
exclude = set(string.punctuation)
lines.english = lines.english.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.korean = lines.korean.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [83]:
from string import digits
remove_digits = str.maketrans('', '', digits)
lines.english = lines.english.apply(lambda x: x.translate(remove_digits))
lines.korean = lines.korean.apply(lambda x: x.translate(remove_digits))

In [84]:
lines.sample(10)

,english,korean
587,keep tom there,톰은 여기에 두세요
3265,is there anything youd like to tell me,뭔가 하고 싶은 얘기 없어
2972,tom bought this horse at an auction,톰은 이 말을 경매에서 샀어
2430,music is an artCOMMA not a science,음악은 예술이지COMMA 과학이 아니야
1969,whats your skype username,네 스카이프 사용자 이름은 뭐야
1385,someone stole my watch,누군가 내 시계를 훔쳤어
2143,her faith in god is unshaken,그녀의 신앙심은 흔들림이 없다
1747,i think you both know tom,너희들 둘 다 톰을 아는 것 같은데
1945,tom often sings in his car,톰은 자기 차 안에서 가끔 노래해
122,fantastic,끝내주네


In [85]:
# applying start and end tokens in french sentences
lines.korean = lines.korean.apply(lambda x: 'START_' + ' ' + x + ' ' + '_END')
lines.head()

,english,korean
0,go,START_ 가 _END
1,hi,START_ 안녕 _END
2,run,START_ 뛰어 _END
3,run,START_ 뛰어 _END
4,who,START_ 누구 _END


In [86]:
# collecting all unique english words to create a vocabulary
all_english_words = set()
for eng in lines.english:
  for word in eng.split():
    if word not in all_english_words:
      all_english_words.add(word)

# collecting all unique french words to create a vocabulary
all_korean_words = set()
for kre in lines.korean:
  for word in kre.split():
    if word not in all_korean_words:
      all_korean_words.add(word)

In [87]:
# printing length of words in each language
print('length of english words: ', len(all_english_words))
print('length of korean words: ', len(all_korean_words))

length of english words:  2707
length of korean words:  5816


In [88]:
# getting maximum sentence length of english sentences
length_list = []
for l in lines.english:
  length_list.append(len(l.split(' ')))

max_input_length = np.max(length_list)
print('max_input_length: ', max_input_length)

max_input_length:  101


In [89]:
# getting maximum sentence length of french sentences
length_list = []
for l in lines.korean:
  length_list.append(len(l.split(' ')))

max_output_length = np.max(length_list)
print('max_output_length: ', max_output_length)

max_output_length:  91


tokenization

In [90]:
# making a list of all input and output words and sorting them out
input_words = sorted(list(all_english_words))
output_words = sorted(list(all_korean_words))
print('all input words: ', input_words)
print('all output words: ', output_words)

#getting total tokens(words) from input and output
num_encoder_tokens = len(all_english_words)
num_decoder_tokens = len(all_korean_words)
print('encoder tokens: ', num_encoder_tokens)
print('decoder tokens: ', num_encoder_tokens)

all input words:  ['COMMA', 'COMMA°c', 'a', 'able', 'aboard', 'about', 'above', 'abroad', 'abroadCOMMA', 'absolutely', 'abusive', 'accept', 'accident', 'accomplishments', 'account', 'accurate', 'acquainted', 'act', 'acted', 'acting', 'actinium', 'action', 'actions', 'active', 'actor', 'acts', 'actually', 'actuallyCOMMA', 'acupuncture', 'add', 'addict', 'addicted', 'addictive', 'address', 'adjustments', 'admission', 'adorable', 'adultCOMMA', 'adults', 'advanced', 'advice', 'afraid', 'africa', 'after', 'afternoon', 'again', 'against', 'age', 'ageCOMMA', 'agesCOMMA', 'ago', 'agree', 'agreed', 'ahead', 'aimed', 'air', 'airport', 'alarm', 'alcoholic', 'alice', 'alive', 'all', 'allCOMMA', 'allergic', 'allow', 'allowed', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'amateurs', 'amazing', 'american', 'ammo', 'among', 'an', 'ancient', 'and', 'anger', 'angles', 'angry', 'animal', 'animals', 'announcement', 'annoying', 'another', 'answer', 'answered', 'answering', 'a

In [91]:
# getting index for words as these indexes will behave as words for machine interactions
input_token_index = dict([(word,i) for i,word in enumerate(input_words)])
output_token_index = dict([(word,i) for i,word in enumerate(output_words)])

print('input token index: ', input_token_index)
print('output token index: ', output_token_index)

input token index:  {'COMMA': 0, 'COMMA°c': 1, 'a': 2, 'able': 3, 'aboard': 4, 'about': 5, 'above': 6, 'abroad': 7, 'abroadCOMMA': 8, 'absolutely': 9, 'abusive': 10, 'accept': 11, 'accident': 12, 'accomplishments': 13, 'account': 14, 'accurate': 15, 'acquainted': 16, 'act': 17, 'acted': 18, 'acting': 19, 'actinium': 20, 'action': 21, 'actions': 22, 'active': 23, 'actor': 24, 'acts': 25, 'actually': 26, 'actuallyCOMMA': 27, 'acupuncture': 28, 'add': 29, 'addict': 30, 'addicted': 31, 'addictive': 32, 'address': 33, 'adjustments': 34, 'admission': 35, 'adorable': 36, 'adultCOMMA': 37, 'adults': 38, 'advanced': 39, 'advice': 40, 'afraid': 41, 'africa': 42, 'after': 43, 'afternoon': 44, 'again': 45, 'against': 46, 'age': 47, 'ageCOMMA': 48, 'agesCOMMA': 49, 'ago': 50, 'agree': 51, 'agreed': 52, 'ahead': 53, 'aimed': 54, 'air': 55, 'airport': 56, 'alarm': 57, 'alcoholic': 58, 'alice': 59, 'alive': 60, 'all': 61, 'allCOMMA': 62, 'allergic': 63, 'allow': 64, 'allowed': 65, 'almost': 66, 'alone

In [93]:
# creating arrays of input and output data
encoder_input_data = np.zeros((len(lines.english), max_input_length), dtype='float32')
decoder_input_data = np.zeros((len(lines.korean), max_output_length), dtype='float32')

#one hot encoding the target data as Dense layer only gives one output through softmax layer
decoder_target_data = np.zeros((len(lines.korean), max_output_length, num_decoder_tokens))

MemoryError: Unable to allocate 15.1 GiB for an array with shape (3833, 91, 5816) and data type float64

In [94]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(3833, 101)
(3833, 91)
(3833, 91, 5816)


In [95]:
# putting all the integer values in input, output data and target data
for i,(input_text, output_text) in enumerate(zip(lines.english, lines.korean)):
  for t, word in enumerate(input_text.split()):
    encoder_input_data[i,t] = input_token_index[word]
  for t,word in enumerate(output_text.split()):
    decoder_input_data[i,t] = output_token_index[word]
    # as decoder target data is ahead of decoder input data, it will not include start_ character(which will be given to decoder model at prediction)
    if t > 0:
      decoder_target_data[i,t-1,output_token_index[word]] = 1

In [96]:
print("encoder input data: ", encoder_input_data[1])
print('decoder input data: ', decoder_input_data[1])
print('decoder target data: ',decoder_target_data[1])
print('shape of sample decoder target data: ', decoder_target_data[1].shape)

encoder input data:  [1094.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.]
decoder input data:  [   9. 3330.   14.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.  

create and training the model

In [97]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [98]:
# setting hyperparameters
embedding_size = 120
lstm_dim = 324

In [99]:
# building model for training stage
#encoder model

encoder_inputs = Input(shape=(None,))
en_x = Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(lstm_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
encoder_states = [state_h, state_c]

In [100]:
# decoder model

decoder_inputs = Input(shape=(None,))
final_dex = Embedding(num_decoder_tokens, embedding_size)(decoder_inputs)

decoder_lstm = LSTM(lstm_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax') 

decoder_outputs = decoder_dense(decoder_outputs)

In [101]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [102]:
model.compile(optimizer='rmsprop',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [103]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [104]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [105]:
model.summary()
plot_model(model, to_file='Model_plot.png', show_shapes=True, show_layer_names=True)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, None, 120)    324840      ['input_5[0][0]']                
                                                                                                  
 embedding_4 (Embedding)        (None, None, 120)    697920      ['input_6[0][0]']                
                                                                                            

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 120)    324840      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 120)    697920      ['input_2[0][0]']                
                                                                                              

In [32]:
r = model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=16, epochs=10, validation_split=0.10)

Epoch 1/10
216/216 [==============================] - 1413s 6s/step - loss: 0.3870 - accuracy: 0.0122 - val_loss: 0.7165 - val_accuracy: 0.0128
Epoch 2/10
216/216 [==============================] - 1974s 9s/step - loss: 0.3757 - accuracy: 0.0126 - val_loss: 0.7391 - val_accuracy: 0.0128
Epoch 3/10
216/216 [==============================] - 1625s 8s/step - loss: 0.3726 - accuracy: 0.0126 - val_loss: 0.7608 - val_accuracy: 0.0128
Epoch 4/10
216/216 [==============================] - 1125s 5s/step - loss: 0.3705 - accuracy: 0.0126 - val_loss: 0.7819 - val_accuracy: 0.0128
Epoch 5/10
216/216 [==============================] - 1363s 6s/step - loss: 0.3692 - accuracy: 0.0126 - val_loss: 0.7933 - val_accuracy: 0.0128
Epoch 6/10
216/216 [==============================] - 1274s 6s/step - loss: 0.3682 - accuracy: 0.0126 - val_loss: 0.8025 - val_accuracy: 0.0128
Epoch 7/10
216/216 [==============================] - 1285s 6s/step - loss: 0.3673 - accuracy: 0.0126 - val_loss: 0.8062 - val_accuracy:

In [106]:
#Inference Stage

#encoder model
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 120)         324840    
                                                                 
 lstm_2 (LSTM)               [(None, 324),             576720    
                              (None, 324),                       
                              (None, 324)]                       
                                                                 
Total params: 901,560
Trainable params: 901,560
Non-trainable params: 0
_________________________________________________________________


In [107]:
#decoder model
decoder_state_input_h = Input(shape=(lstm_dim,))
decoder_state_input_c = Input(shape=(lstm_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2 = Embedding(num_decoder_tokens, embedding_size)(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_state_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs2] + decoder_states2)

In [108]:
# reversing the word index dictionary to get words from index values
reverse_input_char_index = dict((i,char) for char, i in input_token_index.items())
reverse_output_char_index = dict((i,char) for char, i in output_token_index.items())
print(reverse_input_char_index)
print(reverse_output_char_index)

{0: 'COMMA', 1: 'COMMA°c', 2: 'a', 3: 'able', 4: 'aboard', 5: 'about', 6: 'above', 7: 'abroad', 8: 'abroadCOMMA', 9: 'absolutely', 10: 'abusive', 11: 'accept', 12: 'accident', 13: 'accomplishments', 14: 'account', 15: 'accurate', 16: 'acquainted', 17: 'act', 18: 'acted', 19: 'acting', 20: 'actinium', 21: 'action', 22: 'actions', 23: 'active', 24: 'actor', 25: 'acts', 26: 'actually', 27: 'actuallyCOMMA', 28: 'acupuncture', 29: 'add', 30: 'addict', 31: 'addicted', 32: 'addictive', 33: 'address', 34: 'adjustments', 35: 'admission', 36: 'adorable', 37: 'adultCOMMA', 38: 'adults', 39: 'advanced', 40: 'advice', 41: 'afraid', 42: 'africa', 43: 'after', 44: 'afternoon', 45: 'again', 46: 'against', 47: 'age', 48: 'ageCOMMA', 49: 'agesCOMMA', 50: 'ago', 51: 'agree', 52: 'agreed', 53: 'ahead', 54: 'aimed', 55: 'air', 56: 'airport', 57: 'alarm', 58: 'alcoholic', 59: 'alice', 60: 'alive', 61: 'all', 62: 'allCOMMA', 63: 'allergic', 64: 'allow', 65: 'allowed', 66: 'almost', 67: 'alone', 68: 'along', 

In [109]:
# function to predict translation
def decode_seq(input_seq):
  state_values = encoder_model.predict(input_seq)

  target_seq = np.zeros((1,1))

  target_seq[0,0] = output_token_index['START_']

  stop_condition = False
  decoded_sentence = ''

  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + state_values)

    sampled_token_index = np.argmax(output_tokens[0,-1,:])
    sampled_char = reverse_output_char_index[sampled_token_index]

    decoded_sentence += ' ' + sampled_char

    if(sampled_char == '_END' or len(decoded_sentence) >100 ):
      stop_condition = True

    target_seq = np.zeros((1,1))
    target_seq[0,0] = sampled_token_index

    state_values = [h,c] 

  return decoded_sentence

In [110]:
# testing the model for a sample from existing data
for seq_index in [1234, 356, 565, 34, 2345,656]:
  input_seq = encoder_input_data[seq_index:seq_index+1]
  decoded_sentence = decode_seq(input_seq)
  print('----')
  print('Input_sentence: ', lines.english[seq_index:seq_index+1])
  print('decoded sentence: ', decoded_sentence)

1/1 [==============================] - 0s 21ms/step
----
Input_sentence:  1234    dont forget to floss
Name: english, dtype: object
decoded sentence:   돌아올거야 돌아올거야 돌아올거야 야옹하고 야옹하고 가질 가질 만나는 가질 신고 가질 신고 두 인내심을 내야한다 시작한지 해야한다는 필요합니다 파티에서 톰만 전부니 전부니 프랑스어만큼
1/1 [==============================] - 0s 32ms/step
----
Input_sentence:  356    tom drowned
Name: english, dtype: object
decoded sentence:   돌아올거야 돌아올거야 돌아올거야 야옹하고 야옹하고 가질 가질 만나는 가질 신고 가질 신고 두 인내심을 내야한다 시작한지 해야한다는 필요합니다 파티에서 톰만 전부니 전부니 프랑스어만큼
1/1 [==============================] - 0s 20ms/step
----
Input_sentence:  565    do you hear me
Name: english, dtype: object
decoded sentence:   돌아올거야 돌아올거야 돌아올거야 야옹하고 야옹하고 가질 가질 만나는 가질 신고 가질 신고 두 인내심을 내야한다 시작한지 해야한다는 필요합니다 파티에서 톰만 전부니 전부니 프랑스어만큼
1/1 [==============================] - 0s 24ms/step
----
Input_sentence:  34    we try
Name: english, dtype: object
decoded sentence:   돌아올거야 돌아올거야 돌아올거야 야옹하고 야옹하고 가질 가질 만나는 가질 신고 가질 신고 두 인내심을 내야한다 시작한지 해야한다는 필요합니다 파티에서 톰만 전부니 전부니 프랑스어만큼
1/1 [============